#### Imports/setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import os
# import pandas as pd
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from parameters.config_output_naming import (
    geo_id_column, 
    out_file_wide, 
    out_shapefile
)

from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    register_fc_and_append_to_csv,
    # add_geo_ids_to_csv_from_lookup_df,
    add_geo_ids_to_csv_from_lookup_csv,
    add_empty_column_to_csv,
    remove_column_from_csv
)

# from modules.multiband_stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid
)

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out
   

Earth Engine has been initialized with the specified credentials.


#### Set up asset registry

In [3]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : without GEOIDS

GEE asset (feature collection) 

In [10]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'

# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 1#0.0010

In [12]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

roi = fc_sample
print(roi.size().getInfo())

5412


### Whisp it
- If over >500 polygons currently this will go to Google Drive and require manual uploading here
- Stats sent to a separate notebook 'process_whisp_stats.pynb'

In [14]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
Over size limit for on the fly processing. 
 Attempting export to Google Drive 'Whisp_results/Whisp_output_table_export.csv' as a batch task... 


EEException: User memory limit exceeded.

EEException: User memory limit exceeded.

### Display it


In [7]:
df

,system:index,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA,KBA
PLOTID,,,,,,,,,,,,,,,,,,
1,000000000000000007b6,0.4,CIV,100,0,31,100,100,100,100,69,-,0,0,0,54,-,-
2,00000000000000000121,0.7,CIV,100,0,37,100,49,100,100,58,-,0,0,0,3,-,-
3,00000000000000000df1,3.1,CIV,100,0,81,100,91,100,97,19,-,0,0,0,26,-,-
4,0000000000000000029d,4.0,CIV,82,0,22,98,86,98,98,73,-,0,0,0,43,-,-
5,00000000000000000835,4.1,CIV,100,0,52,100,86,100,100,48,-,0,0,0,33,-,-
6,000000000000000009d2,5.4,CIV,63,0,0,100,76,85,93,47,-,0,0,0,3,-,-
7,000000000000000014d6,11.2,CIV,94,0,18,95,63,100,98,73,-,0,0,0,1,-,-


#### Export it to CSV file
NB skip if exporting to Google Drive

In [ ]:
#export output csv
df.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)


#### Add Geo ids
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- In the meantime to stop timeouts extend session length click dollar sign and edit under 'Sessions'.
- If processing doesn't complete, rerun and it should pickup from where you got to.

In [ ]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [ ]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv="whisp_output_table.csv",
        geo_id_lookup_csv="temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        # override_checks=False, # needs implementing
        overwrite=False,
        drop_geo=False,
        debug=True)

##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [ ]:
remove_column_from_csv(
    csv_file="copy_whisp_output_table.csv", # this may change depending on if overwrite is on
    column_name="system:index"
)

#### TO DO: 
    - column order not kept if batch task sent to drive; 
    - fool proof the join to csv (as with the join to fc version, so you know how many were joined etc) etc; 
    - possibly: better checks for lookup creation - check system ids match and prompt to remove old lookup if so
    - possibly: make lookup creation output stay on one line; get system_index printed so can see where errors happen
    - error log for the lookup creation (currently have to check for empty rows)
    